# Preprocessing data
# Create 3 files : hardHam_clean, easyHam_clean, spam_clean

In [1]:
import regex as re
import os
from text_preprocessing import preprocess_text
from text_preprocessing import (
    to_lower,
    remove_email, 
    remove_url, 
    remove_punctuation, 
    lemmatize_word, 
    remove_number, 
    stem_word, 
    remove_stopword,
    remove_whitespace
)
# Thu vien email.parser dung cho tep file co cau truc gioi tin email http
from email.parser import Parser

### Lay duong dan den tat ca cac file trong folder
def read_files_name(folderPath):
    result = []
    for dirPath, dirs, files in os.walk(folderPath):
        for fileName in files:
            fName = os.path.join(dirPath, fileName)
            result.append(fName)
    return result

### Tra ve 3 mang chua danh sach ten file trong thu muc
def read_forders_Path():
    easyHam = read_files_name('dataset\easy_ham')
    hardHam = read_files_name('dataset\hard_ham')
    spam = read_files_name('dataset\spam')
    return easyHam, hardHam, spam


def strip_url(mess):
    return re.sub(r'http[s]?://\S+|www\.\S+', ' ', mess)


def strip_html(mess):
    # clean html_tag
    clean = re.compile('<.*?>')
    mess = re.sub(clean, ' ', mess)
    # clean html_space_white
    return re.sub('&nbsp;', ' ', mess)



def strip_email_header(message):
    message = Parser().parsestr(message)
    subject = message.get('subject', '')
    body = get_email_body(message)

    return '\n'.join((subject, body))

def get_email_body(message):
    payloads = message.get_payload()
    if isinstance(payloads, list):
        return '\n'.join([get_email_body(message) for message in payloads])
    elif isinstance(payloads, str):
        return payloads


def preprocessing_data(mess):
    # processing_function_list = [
    #     str.lower,
    #     strip_url,
    #     remove_url,
    #     strip_html,
    #     remove_email,
    #     remove_number,
    #     remove_stopword,
    #     remove_punctuation,
    #     remove_whitespace

    # ]

#     preprocess_functions = [strip_email_header, to_lower, remove_email, remove_url,strip_url,strip_html, 
#                             remove_number, remove_punctuation, remove_stopword, lemmatize_word]
    preprocess_functions = [to_lower, remove_email, remove_url, 
                            remove_number, remove_punctuation, remove_stopword, lemmatize_word]
    aftermess = preprocess_text(mess, preprocess_functions)

    # return ' '.join(aftermess)
    return aftermess

def handler():
    easyHam,hardHam,spam = read_forders_Path()
    count=0
#     with open(a[0], 'r', encoding='latin') as fi:
#         print(preprocessing_data(fi.read()))
    f=open('dataset\easyHam_clean.txt','a+',encoding='utf-8')
#     print(len(easyHam))
    for i in easyHam:
        try:
            with open(i, 'r', encoding='latin') as fi:
                temp=preprocessing_data(fi.read())
            f.write(temp)
#                print(preprocessing_data(fi.read())) 
            f.write('\n')
        except Exception as e:
            print(i + "Exception :" + str(e))
            count+=1
            continue
    f.close()
    ### hardHam
    f=open('dataset\hardHam_clean.txt','a+',encoding='utf-8')
    for i in hardHam:
        try:
            with open(i, 'r', encoding='latin') as fi:
                temp=preprocessing_data(fi.read())
            f.write(temp)
#                print(preprocessing_data(fi.read())) 
            f.write('\n')
        except Exception as e:
            print(i + "Exception :" + str(e))
            count+=1
            continue
    f.close()
    
    ### spam 
    f=open('dataset\spam_clean.txt','a+',encoding='utf-8')
    for i in spam:
        try:
            with open(i, 'r', encoding='latin') as fi:
                temp=preprocessing_data(fi.read())
            f.write(temp)
#                print(preprocessing_data(fi.read())) 
            f.write('\n')
        except Exception as e:
            print(i + "Exception :" + str(e))
            count+=1
            continue
    f.close()
    print("End Game\n")
#     for i in hardHam:
#         f=open('dataset\hardHam_clean.txt','w+')
#         with open(i, 'r', encoding='latin') as fi:
#             f.write(preprocessing_data(fi.read()))
#         f.write(' ')
#     f.close()
#     for i in spam:
#         f=open('dataset\spam_clean.txt','w+')
#         with open(i, 'r', encoding='latin') as fi:
#             f.write(preprocessing_data(fi.read()))
#         f.write(' ')
#     f.close()
if __name__ == "__main__":
    handler()


End Game



# Thống kê tần suất xuất hiện và tính xác suất trước

In [2]:
# Viết 1 hàm: input 1 file, output 1 mảng kiểu map (từ, fre)
from collections import Counter, defaultdict
from random import shuffle
import pandas as pd
from math import log

In [3]:
def load_data(fileName):
    with open('dataset\\'+fileName, 'r', encoding='utf-8') as fi:
            temp=fi.readlines()
    return temp

In [4]:
def word_freq(data_train):
    result=Counter()
    for email in data_train:
        word=email.split(' ')
        result.update(word)
    return result

In [5]:
# def readData():
#     easyHam = load_data('easyHam_clean.txt')
#     hardHam = load_data('hardHam_clean.txt')
#     spam = load_data('spam_clean.txt')
#     return easyHam,hardHam,spam
data= pd.read_csv("dataset/spam.csv",encoding = 'latin-1')
data.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
data.rename(columns = {'v1':'class_label','v2':'message'},inplace=True)
data.head()
data['length'] = data['message'].apply(len)

In [6]:
data_ham  = data[data['class_label'] == "ham"].copy()
data_spam = data[data['class_label'] == "spam"].copy()

In [7]:
data['class_label'] = data['class_label'].map( {'spam': 1, 'ham': 0})

In [8]:
messages=data['message'].tolist()
messages_clean=[]
for i in messages:
    messages_clean.append(preprocessing_data(i))

In [12]:
print(len(messages_clean))

5572


In [13]:
labels=data['class_label'].tolist()

In [6]:
# easyHam,hardHam,spam=readData()

In [7]:
# # datas=easyHam+hardHam+spam
# datas=hardHam+spam

In [8]:
# # labels = [0] * len(easyHam)+[0]*len(hardHam)+[1]*len(spam)
# labels = [0]*len(hardHam)+[1]*len(spam)

In [15]:
#Hàm trộn theo tỷ lệ cho 1 nhãn, output = train_set,test_set theo nhãn
def shuffle_data(mail_by_label,ratio):
    tmp=mail_by_label.copy()
    shuffle(tmp)
    index=int(len(mail_by_label)*ratio)
#     print (index,len(tmp))
    return tmp[:index],tmp[index:]

In [16]:
# Hàm chia tỷ lệ, mails = data, lables là mảng nhãn ratio là tỷ lệ , input (mails,labels, tỷ lệ), output -> train_set,test_set
# Hàm defaultdict tạo ra kiểu dữ liệu key - value. Trùng key sẽ nối danh sách liên kết vào value
# Hàm zip () :input 2 mảng mails và lables cùng kích thước -> output 1 mảng, mỗi phần tử của mảng gồm 2 phần tử của mail và lable
# Hàm set (): VD input [0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,3,3] -> output [0,1,2,3]
def split_data(mails,labels,ratio):
    split_data_by_labels=defaultdict(list)
    for mail,label in zip(mails,labels):
        split_data_by_labels[label].append(mail)
    train,test=[],[]
    for label in set(labels):
        data_train,data_test=shuffle_data(split_data_by_labels[label],ratio)
        label_train=[label]*len(data_train)
        label_test=[label]*len(data_test)
        train.extend(zip(label_train,data_train))
        test.extend(zip(label_test,data_test))
    data_train = [item[1] for item in train]
    label_train =[item[0] for item in train]
    data_test =[item[1] for item in test]
    label_test =[item[0] for item in test]
    return data_train,data_test,label_train,label_test

In [32]:
# data_train,data_test,label_train,label_test=split_data(datas,labels,0.4)
data_train,data_test,label_train,label_test=split_data(messages_clean,labels,0.8)
print(len(data_train))
print(len(data_test))

4457
1115


In [33]:
#nhóm email theo nhẫn lớp
def split_by_label(mails, labels):
    split_data_by_labels=defaultdict(list)
    for mail,label in zip(mails,labels):
        split_data_by_labels[label].append(mail)
    return split_data_by_labels
# Tính tần suất xuất hiện của từ trong từng nhãn lớp, và số email của từng nhẫn lớp
def create_Dir(data_train,label_train):
    split_data_by_labels = split_by_label(data_train,label_train)
    spam_freq=word_freq(split_data_by_labels[1])
    ham_freq=word_freq(split_data_by_labels[0])
    total_spam_emails = len(split_data_by_labels[1])
    total_ham_emails = len(split_data_by_labels[0])
    return spam_freq,ham_freq,total_spam_emails,total_ham_emails 

In [34]:
#tính xác suất
def cal_prob(data_train,label_train):
    word_in_spam_freq, word_in_ham_freq,total_spam_emails,total_ham_emails = create_Dir(data_train,label_train)
    #số lần xuất hiện của các từ trong từng nhãn lớp
    total_word_in_spam = sum( word_in_spam_freq.values())
    total_word_in_ham = sum( word_in_ham_freq.values())
    #giá trị tuyệt đối của T
    spam_words = set(word_in_spam_freq.keys())
    ham_words = set(word_in_ham_freq.keys())
    dictionary = spam_words.union(ham_words)
    #xác suất trước của P(ci)
    total_emails= total_spam_emails + total_ham_emails
    spam_prob = total_spam_emails/total_emails
    ham_prob = total_ham_emails/total_emails
    #xác suất của từ khóa trong từ điển đối với hai nhãn
    word_in_spam_probs, word_in_ham_probs = Counter(),Counter()
    for  word in dictionary:
        word_in_spam_probs[word] = (word_in_spam_freq[word]+1)/(total_word_in_spam+len(dictionary))
        word_in_ham_probs[word] = (word_in_ham_freq[word]+1)/(total_word_in_ham +len(dictionary))
    data_ham_probs = pd.DataFrame.from_dict(word_in_ham_probs, orient='index').reset_index()
    data_ham_probs = data_ham_probs.rename(columns={'index':'words', 0:'values'})
    data_spam_probs= pd.DataFrame.from_dict(word_in_spam_probs, orient='index').reset_index()
    data_spam_probs=data_spam_probs.rename(columns={'index':'words', 0:'values'})
    data_ham_probs.to_csv('data_ham_probs.csv', index = False, header=True)
    data_spam_probs.to_csv('data_spam_probs.csv', index = False, header=True) 
    df_pre_prob= pd.DataFrame( {'spam_prob':[spam_prob],'ham_prob':[ham_prob]})
    df_pre_prob.to_csv('data_pre_probs.csv', index = False, header=True)
    print(df_pre_prob)
#     return spam_prob, ham_prob, word_in_spam_probs, word_in_ham_probs
    
cal_prob(data_train,label_train)


   spam_prob  ham_prob
0   0.133947  0.866053


In [35]:
def load_model():
    data_ham_probs= pd.read_csv("data_ham_probs.csv") 
    data_spam_probs= pd.read_csv("data_spam_probs.csv")
    data_pre_prob= pd.read_csv("data_pre_probs.csv")
    # lấy xác suất từ files
    word_in_ham_probs= data_ham_probs.set_index('words')['values'].to_dict()
    word_in_spam_probs= data_spam_probs.set_index('words')['values'].to_dict()
    spam_pre_prob = data_pre_prob['spam_prob'][0]
    ham_pre_prob = data_pre_prob['ham_prob'][0]
    return  word_in_ham_probs, word_in_spam_probs, spam_pre_prob, ham_pre_prob
def classify(mail, word_in_ham_probs, word_in_spam_probs, spam_pre_prob, ham_pre_prob):

#     # lấy tập từ
#     spam_words = set(word_in_spam_probs.keys())
#     ham_words = set(word_in_ham_probs.keys())
#     dictionary = spam_words.union(ham_words)
    #tính xác suất
    mail=preprocessing_data(mail)
    mail_words = mail.split(' ')
    prob_spam = prob_ham = 0.0
    for mail_word in set(mail_words):
        if(word_in_ham_probs.get(mail_word) is not None):
            prob_ham += log(word_in_ham_probs.get(mail_word))
        if(word_in_spam_probs.get(mail_word) is not None):
            prob_spam += log(word_in_spam_probs.get(mail_word))
    prob_ham += log( ham_pre_prob)
    prob_spam += log( spam_pre_prob)

    if prob_spam > prob_ham:
        return 1
    return 0
    

In [41]:
def pre_mails(data_test):    
    result=[]
    word_in_ham_probs, word_in_spam_probs, spam_pre_prob, ham_pre_prob = load_model()
    for i in range(len(data_test)):
        result.append(classify(data_test[i], word_in_ham_probs, word_in_spam_probs, spam_pre_prob, ham_pre_prob))
    return result

In [51]:
predicted_labels=pre_mails(data_test)

In [48]:
def accuracy(label_test,predicted_labels):
    correct = sum([1 if label_test == predicted_label else 0
                    for label_test, predicted_label in zip(label_test, predicted_labels)])
    return correct / len(label_test)*100

In [49]:
print(accuracy(label_test,predicted_labels))

97.39910313901345


In [53]:
def confusion_matrix(label_test,predicted_labels, label):
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for label_test, predicted_label in zip(label_test, predicted_labels):
        if label_test == label and predicted_label == label:
            true_positive += 1
        elif label_test != label and predicted_label != label:
            true_negative += 1
        elif label_test == label and predicted_label != label:
            false_negative += 1
        else:
            false_positive += 1
    return (true_positive, false_negative, false_positive, true_negative)

In [54]:
true_positive_0,false_negative_0,false_positive_0,true_negative_0=confusion_matrix(label_test,predicted_labels,0)
true_positive_1,false_negative_1,false_positive_1,true_negative_1=confusion_matrix(label_test,predicted_labels,1)

In [55]:
Precision_0=true_positive_0/(true_positive_0+false_positive_0)
Precision_1=true_positive_1/(true_positive_1+false_positive_1)
Recall_0=true_positive_0/(true_positive_0+false_negative_0)
Recall_1=true_positive_1/(true_positive_1+false_negative_1)

In [56]:
Precision_0

0.9824742268041237

In [57]:
Precision_1

0.9172413793103448

In [58]:
Recall_0

0.9875647668393782

In [59]:
Recall_1

0.8866666666666667

In [60]:
F1_0=(2*Precision_0*Recall_0)/(Precision_0+Recall_0)
F1_1=(2*Precision_1*Recall_1)/(Precision_1+Recall_1)

In [61]:
print(F1_0)
print(F1_1)

0.9850129198966409
0.9016949152542374
